In [4]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import RepeatedKFold, train_test_split
import optuna

In [5]:
dft = pd.read_csv("train.csv")
tdf = pd.read_csv("test.csv")

In [ ]:
print(dft.info())
print("--------")
print(tdf.info())

In [ ]:
def check_null(df):
    for col in df.columns:
        print(f"{col}: {sum(dft[col].isna())}")
check_null(dft)
print("---------")
check_null(tdf)

In [ ]:
print(dft.nunique())
print("-------")
print(tdf.nunique())

In [9]:
cat_cols = ['Gender', 'City_Category', 'Customer_Category']

dfu = dft.drop(columns=['Product_Holding_B1', 'Product_Holding_B2', 'Customer_ID'])
dfu = pd.get_dummies(dfu, columns=cat_cols, prefix=cat_cols).astype(np.float32)

tdu = tdf.drop(columns=['Product_Holding_B1', 'Customer_ID'])
tdu = pd.get_dummies(tdu, columns=cat_cols, prefix=cat_cols).astype(np.float32)

In [ ]:
from ast import literal_eval as le
def interact(ser):
    nop = 22
    nou = ser.shape[0]
    int_arr = np.zeros((nou, nop), dtype=np.float32)
    for i in range(nou):
        lst = le(ser[i])
        for prod in lst:
            idx = int(prod[1:])
            int_arr[i][idx] = 1
    
    print(int_arr.shape)
    return int_arr


dfi = interact(dft['Product_Holding_B1'])
dfr = interact(dft['Product_Holding_B2'])
tdi = interact(tdf['Product_Holding_B1'])

In [11]:
# check for any issues through shapes

assert dfi.shape[0]==dfu.shape[0]
assert dfi.shape[1]==22
assert dfr.shape[0]==dfu.shape[0]
assert dfr.shape[1]==22
assert tdi.shape[0]==tdf.shape[0]
assert tdi.shape[1]==22
assert dfu.shape[1]==10
assert tdu.shape[1]==10

In [ ]:
# prep train data

X = np.concatenate((dfu.to_numpy(dtype=np.float32), dfi), axis=1)
Y = dfi + dfr
assert X.shape[0]==Y.shape[0]
print(X.shape)
print(Y.shape)

---
### Modelling

In [13]:
def get_model(trial, n_inputs, n_outputs):
    model = Sequential()
    n_layers = trial.suggest_int('nlayers', 1, 6)
    num_hidden = trial.suggest_int('n_didden__input', 10, 129, step=2)
    model.add(Dense(num_hidden, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    
    for layer_i in range(n_layers):
        num_hidden = trial.suggest_int(f'n_hidden_{layer_i}', 10, 129, step=2)
        model.add(Dense(num_hidden, activation='relu'))
        drop = trial.suggest_float(f'drop_{layer_i}', 0, 0.6)
        model.add(Dropout(drop))
    
    model.add(Dense(n_outputs, activation='sigmoid'))
    
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    optim = keras.optimizers.Adam(learning_rate=lr)
    prec = keras.metrics.Precision()
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=[prec])
    
    return model

In [14]:
def objective(trial):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=21)
    
    model = get_model(trial, X.shape[1], Y.shape[1])
    n_epochs = trial.suggest_int('n_epochs', 10, 50)
    bs = trial.suggest_categorical('batch_size', [16, 32])
    vs = trial.suggest_categorical('split_size', [0.2, 0.3])
    model.fit(X, Y, batch_size=bs, epochs=n_epochs, validation_split=vs, verbose=1, workers=4)
    
    scores = model.evaluate(x_test, y_test, batch_size=16, workers=4)
    return scores[1]

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, n_jobs=-1)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
print(study.best_trials)
print("----------")
print(study.best_value)
print("----------")
print(study.best_params)
print("----------")

In [19]:
# creating optimized model
n_inputs = X.shape[1]
n_outputs = Y.shape[1]
op_model = Sequential()
op_model.add(Dense(64, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
op_model.add(Dense(48, activation='relu'))
op_model.add(Dropout(0.4))
op_model.add(Dense(16, activation='relu'))
op_model.add(Dropout(0.2))
    
op_model.add(Dense(n_outputs, activation='sigmoid'))
    
lr = 0.0003
optim = keras.optimizers.Adam(learning_rate=lr)
prec = keras.metrics.Precision()
op_model.compile(loss='binary_crossentropy', optimizer=optim, metrics=[prec])

In [ ]:
n_epochs = 50
bs = 16
vs = 0.2
op_model.fit(X, Y, batch_size=bs, epochs=n_epochs, validation_split=vs, verbose=1, workers=4)

In [ ]:
# Generating predictions

xt = np.concatenate((tdu.to_numpy(dtype=np.float32), tdi), axis=1)
yt = op_model.predict(xt, batch_size=16)
print(yt.shape)

In [ ]:
req_pred = yt - (yt*tdi)
req_pred

In [31]:
# Generating submissions
sub = pd.read_csv("submission.csv")
preds = []
for i in range(req_pred.shape[0]):
  ind = np.argpartition(req_pred[i], -3)[-3:]
  temp = []
  for idx in range(3):
    st = 'P00' if ind[idx]==0 else f'P{ind[idx]}'
    temp.append(st)
  preds.append(temp)

In [ ]:
sub

In [ ]:
preds

In [ ]:
sub['Product_Holding_B2'] = preds
sub

In [35]:
sub.to_csv('sub.csv', index=False)